## Pogramación Lineal
### Arbitraje de Opciones

Este cuaderno muestra un ejemplo de como verificar si es posible realizar arbitraje en una cadena
de opciones call de un mismo vencimiento. Para ello se construye un modelo de programación lineal
que calcula el coste de construir una cartera que:
- de beneficio no negativo si el precio es 0
- de beneficio no negativo para cada uno de los strikes de la cadena
- de beneficio no negativo a partir del último strike (pendiente positiva)

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp

In [ ]:
def call_value(K, S):
    """
    calcula el beneficio de la call a vencimiento.
    Si el precio de la accion es mayor que el de ejercicio
    el beneficio es la diferencia.  Si el precio es menor
    vence sin valor
    """
    p = 0
    if S > K:
        p = S - K
    return p

In [ ]:
print(call_value(100, 105))
print(call_value(100, 95))

### Datos
Presentamos como ejemplo un diccionario que representa la prima para cada precio de ejercicio
en una cadena de opciones 

In [ ]:
strike_call_prices ={
    80: 20.14,
    85: 15.47,
    90: 11.23,
    95: 7.64,
    100: 4.85,
    105: 2.86,
    110: 1.58,
    115: 0.81,
    120: 0.39
}

In [ ]:
# numero de ejercicios
n = len(strike_call_prices)
n

In [ ]:
# array con los precios
call_prices = np.array(list(strike_call_prices.values()))
call_prices

In [ ]:
# array con los precios de ejercicio
call_strikes = list(strike_call_prices.keys())
call_strikes

### Modelo

Las variables de decisión representan el número de contratos que compraremos (positivo)
o venderemos (negativo) para nuestra cartera

In [ ]:
# variables de decision
call_x = cp.Variable(n)

In [ ]:
# precio de la cartera primas x contratos que queremos minimizar
objetivo = cp.sum(cp.multiply(call_x, call_prices))

In [ ]:
#lista de restricciones
constraints = []

(1) añadir restricciones del valor a vencimiento si el precio es 0

In [ ]:
value_at0 = np.zeros(n)
for i, i_strike in enumerate(call_strikes):
    value_at0[i] = call_value(i_strike, 0)
print(value_at0)
exp_at0 = cp.sum(cp.multiply(value_at0, call_x)) >= 0
constraints.append(exp_at0)

(2) añadir restricciones del valor a vencimiento para cada strike

In [ ]:
for j_price in call_strikes:
    value_ati = np.zeros(n)
    for i, i_strike in enumerate(call_strikes):
        value_ati[i] = call_value(i_strike, j_price)
    print(i_strike, "->", value_ati)  
    ati_exp = cp.sum(cp.multiply(value_ati,call_x)) >= 0
    constraints.append(ati_exp)

(3) añadir restricciones de la pendiente del ultimo strike

In [ ]:
top_strike = call_strikes[-1]
value_attop = np.zeros(n)
for i, i_strike in enumerate(call_strikes):
    value_attop[i] = call_value(i_strike, top_strike + 1) - call_value(i_strike, top_strike)
print(value_attop)

exp_top = cp.sum(cp.multiply(value_attop, call_x)) >= 0
constraints.append(exp_top)

Se restringe el numero de contratos vendidos. En la práctica estaríamos limitados
por el control de riesgo de nuestra cuenta.  En el caso hipotético de no restringirse
podríamos vender todo lo que quisieramos cuando haya arbitraje

In [ ]:
constraints.append(call_x >= -40) 

Solucionamos el problema
- resultado 0 significa que no existe arbitraje. La cartera mas barata sería equivalente a no comprar nada
- resultado negativo significa que en lugar de pagar recibimos dinero por nuestra cartera

In [ ]:
problem = cp.Problem(cp.Minimize(objetivo), constraints)
result = problem.solve()

In [ ]:
np.round(result, 4)

compra venta de contratos que tendríamos que realizar

In [ ]:
res_serie = pd.Series(np.round(call_x.value), index=call_strikes)
res_serie

___


### Ejercicio Propuesto

Ejecutar el modelo del ejemplo considerando ahora unos precios de prima alternativos. 
Verificar que en este caso sí existe arbitraje. ¿Qué cartera tendrías que construir?

In [ ]:
strike_call_prices ={
    80: 20.14,
    85: 15.47,
    90: 11.23,
    95: 7.75,
    100: 5.03,
    105: 2.86,
    110: 1.05,
    115: 0.81,
    120: 0.39
}